## TO DO
* Reproduce age prediction
* Reduce data (sampling resolution + precision)
* Reproduce again
* Clean up code

### reproducing CNP_func_preprocess_corr_matrix
https://github.com/danielmlow/brain_age/blob/master/CNP_func_preprocess_corr_matrix.ipynb

In [5]:
import pandas as pd
import numpy as np
import nibabel as nib
import datetime
import os
from glob import glob

In [51]:
toy = False
run_all = True
path = '/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/'
output_dir = '/data1/users/jvogel/nilearn_tutorial/output/'
input_dir = '/data1/users/jvogel/nilearn_tutorial/'

n_jobs = 2 #amount of cores #orignally 20
cv=4
description = 'CNP_func_gridsearch_age_'
log_file = description+datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [52]:
log_file

'CNP_func_gridsearch_age_18-09-21-14-29-21'

In [53]:
# Load subject ids
scandirs = sorted(glob(os.path.join(path,'*/')))
subids = [x.split('/')[-2] for x in scandirs]
subids

['sub-10159',
 'sub-10171',
 'sub-10189',
 'sub-10193',
 'sub-10206',
 'sub-10217',
 'sub-10225',
 'sub-10227',
 'sub-10228',
 'sub-10235',
 'sub-10249',
 'sub-10269',
 'sub-10271',
 'sub-10273',
 'sub-10274',
 'sub-10280',
 'sub-10290',
 'sub-10292',
 'sub-10304',
 'sub-10316',
 'sub-10321',
 'sub-10325',
 'sub-10329',
 'sub-10339',
 'sub-10340',
 'sub-10345',
 'sub-10347',
 'sub-10356',
 'sub-10361',
 'sub-10365',
 'sub-10376',
 'sub-10377',
 'sub-10388',
 'sub-10429',
 'sub-10438',
 'sub-10440',
 'sub-10448',
 'sub-10455',
 'sub-10460',
 'sub-10471',
 'sub-10478',
 'sub-10487',
 'sub-10492',
 'sub-10506',
 'sub-10517',
 'sub-10523',
 'sub-10524',
 'sub-10525',
 'sub-10527',
 'sub-10530',
 'sub-10557',
 'sub-10565',
 'sub-10570',
 'sub-10575',
 'sub-10624',
 'sub-10629',
 'sub-10631',
 'sub-10638',
 'sub-10668',
 'sub-10672',
 'sub-10674',
 'sub-10678',
 'sub-10680',
 'sub-10686',
 'sub-10692',
 'sub-10696',
 'sub-10697',
 'sub-10704',
 'sub-10707',
 'sub-10708',
 'sub-10719',
 'sub-

In [54]:
len(subids)

265

Okay that's weird.. seems we have about 100 more subjects than they do...
Later on they do a train-test split -- could they have already left out 100 subjects before this notebook? 
Need to check this.

Okay figured it out. They are using only controls. Need to separate the controls...

In [55]:
df = pd.read_csv('/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.4/participants.tsv',
                    sep = '\t')
df.head()

,participant_id,diagnosis,age,gender,bart,bht,dwi,pamenc,pamret,rest,scap,stopsignal,T1w,taskswitch,ScannerSerialNumber
0,sub-10159,CONTROL,30,F,1,n/a,1,n/a,n/a,1,1,1,1,1,35343.0
1,sub-10171,CONTROL,24,M,1,1,1,n/a,n/a,1,1,1,1,1,35343.0
2,sub-10189,CONTROL,49,M,1,n/a,1,n/a,n/a,1,1,1,1,1,35343.0
3,sub-10193,CONTROL,40,M,1,n/a,1,n/a,n/a,n/a,n/a,n/a,1,n/a,35343.0
4,sub-10206,CONTROL,21,M,1,n/a,1,n/a,n/a,1,1,1,1,1,35343.0


In [56]:
control_ids = df[df.diagnosis=='CONTROL']['participant_id'].values

In [57]:
len(control_ids)

130

still a disparity but at least we're a lot closer now...

In [58]:
y_age = df.loc[df.participant_id.isin(control_ids),'age'].values
y_age

array([30, 24, 49, 40, 21, 33, 35, 31, 40, 22, 28, 38, 41, 30, 43, 27, 48,
       49, 25, 23, 29, 34, 29, 22, 45, 32, 25, 25, 43, 42, 24, 42, 49, 50,
       25, 36, 25, 26, 37, 25, 40, 30, 21, 31, 22, 21, 25, 21, 24, 46, 24,
       24, 23, 26, 25, 32, 23, 28, 31, 21, 25, 25, 29, 42, 24, 22, 26, 28,
       25, 39, 32, 28, 25, 23, 22, 23, 41, 38, 36, 35, 26, 47, 41, 23, 23,
       21, 25, 44, 37, 25, 22, 27, 30, 45, 24, 27, 32, 47, 26, 49, 47, 40,
       40, 48, 26, 22, 45, 49, 37, 23, 39, 22, 25, 21, 24, 43, 39, 21, 27,
       27, 21, 29, 25, 40, 28, 38, 24, 28, 32, 25])

In [59]:
confounds[0]

'/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10304/func/sub-10304_task-rest_bold_confounds.tsv'

In [60]:
filenames = sorted(glob(os.path.join(path,'sub-*/func/*MNI*preproc.nii.gz')))
filenames = [x for x in filenames if x.split('/')[-1].split('_')[0] in control_ids]
confounds = sorted(glob(os.path.join(path,'sub-*/func/*confounds.tsv')))
confounds = [x for x in confounds if x.split('/')[-1].split('_')[0] in control_ids]
print(len(filenames), len(confounds))

122 122


Bingo

Need to clean up the disparity between y_age and filenames

In [64]:
good_ids = [x.split('/')[-1].split('_')[0] for x in filenames if x.split('/')[-1].split('_')[0] in subj]
extras = [x for x in control_ids if x not in good_ids]
sorted(extras)

['sub-10193',
 'sub-10299',
 'sub-10428',
 'sub-10501',
 'sub-10948',
 'sub-10971',
 'sub-11082',
 'sub-11121']

In [71]:
jnk = control_ids.tolist()
[jnk.remove(x) for x in extras]
final_ids = jnk
len(final_ids)

122

In [24]:
from nilearn import datasets

parcellations = datasets.fetch_atlas_basc_multiscale_2015(version='sym')

# We show here networks of 64, 197, 444
networks_64 = parcellations['scale064']
networks_197 = parcellations['scale197']
networks_444 = parcellations['scale444']


Dataset created in /home/users/jvogel/nilearn_data/basc_multiscale_2015



Downloaded 193985 of 193985 bytes (100.0%,    0.0s remaining) ...done. (2 seconds, 0 min)
Extracting data from /home/users/jvogel/nilearn_data/basc_multiscale_2015/3cbcf0eeb3f666f55070aba1db9a758f/1861819..... done.


In [72]:
output_dir

'/data1/users/jvogel/nilearn_tutorial/output/'

In [73]:
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure, sym_matrix_to_vec

def func_2_corr_matrix(parcellation, name):
    masker = NiftiLabelsMasker(labels_img=parcellation, standardize=True,
                               memory='nilearn_cache', verbose=0)
    #n_regions = 197
    
    correlations = []
    upper_triang_corrs = []
    # Initializing ConnectivityMeasure object with kind='correlation'
    connectome_measure = ConnectivityMeasure(kind='correlation')
    for filename in filenames:
        timeseries_each_subject = masker.fit_transform(filename)
        # call fit_transform from ConnectivityMeasure object
        correlation = connectome_measure.fit_transform([timeseries_each_subject])
        # saving each subject correlation to correlations
        correlations.append(correlation)
        # get the upper triangular correlation matrix of correlation
        upper_triang_corr = sym_matrix_to_vec(correlation, discard_diagonal=True)
        # saving each subject upper_triang_corr to upper_triang_corrs
        upper_triang_corrs.append(upper_triang_corr) 
        print(filename,'finished')
    #   Split train test sets and save
    X_train = upper_triang_corrs[:84]
    X_test = upper_triang_corrs[84:]
    y_train = y_age[:84]
    y_test = y_age[84:]
    random_subj_train = subj[:84]
    random_subj_test = subj[84:]
    np.savez_compressed(output_dir+log_file+'CNP_func'+name+'_train_set',a=X_train, b=y_train, c=random_subj_train)
    np.savez_compressed(output_dir+log_file+'CNP_func'+name+'_test_set',a=X_test, b=y_test, c=random_subj_test)
    return correlations, upper_triang_corrs

correlations197, upper_triang_corrs197 = func_2_corr_matrix(networks_197, '197')
correlations444, upper_triang_corrs444 = func_2_corr_matrix(networks_444, '444')

/home/users/jvogel/anaconda2/envs/py3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:462: UserWarning: The behaviour of affine_transform with a one-dimensional array supplied for the matrix parameter has changed in scipy 0.18.0.
  "The behaviour of affine_transform with a one-dimensional "


/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10159/func/sub-10159_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10171/func/sub-10171_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10189/func/sub-10189_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10206/func/sub-10206_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10217/func/sub-10217_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10225/func/sub-10225_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvo

/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10565/func/sub-10565_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10570/func/sub-10570_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10575/func/sub-10575_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10624/func/sub-10624_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10629/func/sub-10629_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10631/func/sub-10631_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvo

/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-11061/func/sub-11061_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-11062/func/sub-11062_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-11066/func/sub-11066_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-11067/func/sub-11067_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-11068/func/sub-11068_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-11077/func/sub-11077_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvo

/home/users/jvogel/anaconda2/envs/py3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:462: UserWarning: The behaviour of affine_transform with a one-dimensional array supplied for the matrix parameter has changed in scipy 0.18.0.
  "The behaviour of affine_transform with a one-dimensional "


/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10159/func/sub-10159_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10171/func/sub-10171_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10189/func/sub-10189_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10206/func/sub-10206_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10217/func/sub-10217_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10225/func/sub-10225_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvo

/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10565/func/sub-10565_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10570/func/sub-10570_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10575/func/sub-10575_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10624/func/sub-10624_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10629/func/sub-10629_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10631/func/sub-10631_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvo

/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-11061/func/sub-11061_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-11062/func/sub-11062_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-11066/func/sub-11066_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-11067/func/sub-11067_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-11068/func/sub-11068_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-11077/func/sub-11077_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvo

In [79]:
loaded = np.load(os.path.join(output_dir,log_file+'CNP_func197_train_set.npz'))
X_train, y_train, random_subj_train = loaded['a'], loaded['b'], loaded['c']


In [83]:
y_train.shape

(84,)